## Conexión de APIs con Bases de Datos Relacionales (SQLite) 💾🔗

### Introducción

En la clase anterior, construimos una API REST con Flask que gestionaba una lista de tareas. Sin embargo, esas tareas se almacenaban en una lista en memoria, lo que significa que se perdían cada vez que la aplicación Flask se detenía o reiniciaba. Para que los datos sean **persistentes**, necesitamos almacenarlos en una base de datos.

**SQLite** es un sistema de gestión de bases de datos relacionales (RDBMS) autónomo, sin servidor, de configuración cero y transaccional. Es ideal para:
* Desarrollo y pruebas.
* Aplicaciones pequeñas a medianas.
* Situaciones donde no se necesita un servidor de base de datos dedicado.

Python tiene soporte incorporado para SQLite a través del módulo `sqlite3`. En esta clase, modificaremos nuestra API de tareas para que utilice una base de datos SQLite para almacenar y recuperar las tareas.

### 1. Prerrequisitos

* Flask instalado (`pip install Flask`).
* El módulo `sqlite3` viene incluido con la instalación estándar de Python, por lo que no se necesita instalación adicional.

---

## 2. Conceptos Básicos de SQLite y el Módulo `sqlite3` de Python

Antes de integrarlo con Flask, repasemos cómo interactuar con SQLite usando Python.

#### a. Conexión a la Base de Datos
Se crea una conexión a un archivo de base de datos. Si el archivo no existe, SQLite lo creará.

In [1]:
import sqlite3

# conn = sqlite3.connect('mi_base_de_datos.db') # Conecta o crea la base de datos
# print("Conexión a SQLite establecida (o base de datos creada).")
# conn.close() # Siempre cierra la conexión cuando termines

#### b. Creación de un Cursor y Ejecución de SQL
El objeto cursor te permite ejecutar comandos SQL.

In [ ]:
conn = sqlite3.connect('mi_base_de_datos.db')
cursor = conn.cursor()

# Crear una tabla (DDL - Data Definition Language)
cursor.execute('''
 CREATE TABLE IF NOT EXISTS usuarios (
     id INTEGER PRIMARY KEY AUTOINCREMENT,
     nombre TEXT NOT NULL,
     email TEXT UNIQUE NOT NULL
 );
 ''')
print("Tabla 'usuarios' creada o ya existente.")

# Insertar datos (DML - Data Manipulation Language)
# Usar '?' como placeholders para prevenir inyección SQL
try:
     cursor.execute("INSERT INTO usuarios (nombre, email) VALUES (?, ?)", ('Alice Wonderland', 'alice@example.com'))
     cursor.execute("INSERT INTO usuarios (nombre, email) VALUES (?, ?)", ('Bob The Builder', 'bob@example.com'))
     conn.commit() # Guardar (confirmar) los cambios
     print("Datos insertados.")
except sqlite3.IntegrityError as e: # Por si el email ya existe (UNIQUE constraint)
     print(f"Error de integridad al insertar: {e}")

# Consultar datos (DQL - Data Query Language)
cursor.execute("SELECT * FROM usuarios")
todos_los_usuarios = cursor.fetchall() # Obtiene todas las filas
print("\nUsuarios en la base de datos:")
for usuario in todos_los_usuarios:
     print(usuario) # Las filas son tuplas por defecto

# Obtener filas como diccionarios (más conveniente para APIs)
conn.row_factory = sqlite3.Row # Configurar para que las filas se devuelvan como objetos tipo diccionario
cursor = conn.cursor() # Recrear el cursor después de cambiar row_factory
cursor.execute("SELECT * FROM usuarios WHERE nombre = ?", ('Alice Wonderland',))
alice = cursor.fetchone() # Obtiene una sola fila
if alice:
    print("\nDatos de Alice (como diccionario):")
    print(dict(alice)) # Convertir el objeto Row a un diccionario Python estándar
else:
    print("\nAlice no encontrada.")

conn.close()

Tabla 'usuarios' creada o ya existente.
Datos insertados.

Usuarios en la base de datos:
(1, 'Alice Wonderland', 'alice@example.com')
(2, 'Bob The Builder', 'bob@example.com')

Datos de Alice (como diccionario):
{'id': 1, 'nombre': 'Alice Wonderland', 'email': 'alice@example.com'}


: 

**Puntos Clave del Módulo `sqlite3`:**
* `sqlite3.connect('nombre_archivo.db')`: Conecta (y crea si no existe) la base de datos.
* `conn.cursor()`: Crea un objeto cursor para ejecutar SQL.
* `cursor.execute("SQL", (parametros,))`: Ejecuta una sentencia SQL. Usa `?` para pasar parámetros de forma segura.
* `conn.commit()`: Guarda los cambios realizados por operaciones DML (`INSERT`, `UPDATE`, `DELETE`).
* `cursor.fetchall()`: Recupera todas las filas de una consulta `SELECT`.
* `cursor.fetchone()`: Recupera la siguiente fila de una consulta `SELECT`.
* `conn.row_factory = sqlite3.Row`: Permite acceder a las columnas por nombre (como un diccionario), lo cual es muy útil para APIs JSON.
* `conn.close()`: Cierra la conexión a la base de datos.

---

## 3. Modificando la API de Tareas para Usar SQLite

Ahora, tomaremos nuestra API de tareas anterior (que usaba una lista en memoria) y la adaptaremos para que interactúe con una base de datos SQLite llamada `tasks.db`.

### 3.1. Configuración Inicial y Funciones Auxiliares para la BD

Definiremos el nombre de nuestra base de datos y crearemos una función para obtener una conexión y una función para inicializar la tabla `tasks`.

```python
 Este sería el inicio de nuestro archivo tasks_api_sqlite.py

 import sqlite3
 from flask import Flask, request, jsonify

 DATABASE_NAME = 'tasks.db'

 def get_db_connection():
     """Crea y devuelve una conexión a la base de datos SQLite."""
     conn = sqlite3.connect(DATABASE_NAME)
     conn.row_factory = sqlite3.Row # Para acceder a columnas por nombre
     return conn

 def init_db():
     """Inicializa la base de datos y crea la tabla 'tasks' si no existe."""
     conn = get_db_connection()
     cursor = conn.cursor()
     cursor.execute('''
         CREATE TABLE IF NOT EXISTS tasks (
             id INTEGER PRIMARY KEY AUTOINCREMENT,
             titulo TEXT NOT NULL,
             descripcion TEXT,
             completada BOOLEAN NOT NULL CHECK (completada IN (0, 1)) 
         );
     ''')
     conn.commit()
     conn.close()
     print("Base de datos inicializada y tabla 'tasks' creada/verificada.")

 app = Flask(__name__)

# (Aquí irán los endpoints de la API modificados)

 if __name__ == '__main__':
     init_db() # Asegurarse que la tabla exista al iniciar la app
     app.run(debug=True)
```

### 3.2. Actualización de los Endpoints CRUD

Cada endpoint que antes interactuaba con la lista `tasks` ahora interactuará con la base de datos.

#### a) Crear una Tarea (POST /tasks)

```python
 @app.route('/tasks', methods=['POST'])
 def create_task():
     if not request.json or 'titulo' not in request.json:
         return jsonify({'error': 'El título es requerido'}), 400

     nuevo_titulo = request.json['titulo']
     nueva_descripcion = request.json.get('descripcion', "")
     # SQLite no tiene un tipo BOOLEAN nativo, usa INTEGER 0 (False) y 1 (True)
     nueva_completada = 0 # Las tareas se crean como no completadas por defecto

     conn = get_db_connection()
     cursor = conn.cursor()
     try:
         cursor.execute(
             "INSERT INTO tasks (titulo, descripcion, completada) VALUES (?, ?, ?)",
             (nuevo_titulo, nueva_descripcion, nueva_completada)
         )
         conn.commit()
         nueva_id_tarea = cursor.lastrowid # Obtener el ID de la tarea recién insertada
     except sqlite3.Error as e:
         conn.rollback() # Revertir cambios en caso de error
         return jsonify({'error': f'Error en la base de datos: {e}'}), 500
     finally:
         conn.close()

     # Devolver la tarea creada (opcionalmente, podrías hacer otro SELECT para obtenerla)
     tarea_creada = {'id': nueva_id_tarea, 'titulo': nuevo_titulo, 'descripcion': nueva_descripcion, 'completada': False}
     return jsonify(tarea_creada), 201
```

#### b) Obtener Todas las Tareas (GET /tasks)

```python
 @app.route('/tasks', methods=['GET'])
 def get_tasks():
     conn = get_db_connection()
     cursor = conn.cursor()
     cursor.execute("SELECT * FROM tasks")
     filas_tareas = cursor.fetchall()
     conn.close()
    
     # Convertir las filas (objetos sqlite3.Row) a una lista de diccionarios
     tareas_lista = [dict(row) for row in filas_tareas]
    # Convertir el booleano almacenado como 0/1 a True/False para JSON
     for tarea in tareas_lista:
        tarea['completada'] = bool(tarea['completada'])
        
#     return jsonify({'tasks': tareas_lista})
```

#### c) Obtener una Tarea Específica (GET /tasks/<task_id>)

```python
# @app.route('/tasks/<int:task_id>', methods=['GET'])
# def get_task(task_id):
#     conn = get_db_connection()
#     cursor = conn.cursor()
#     cursor.execute("SELECT * FROM tasks WHERE id = ?", (task_id,))
#     fila_tarea = cursor.fetchone()
#     conn.close()
    
#     if fila_tarea:
#         tarea = dict(fila_tarea)
#         tarea['completada'] = bool(tarea['completada'])
#         return jsonify(tarea)
#     else:
#         return jsonify({'error': 'Tarea no encontrada'}), 404
```

#### d) Actualizar una Tarea (PUT /tasks/<task_id>)

```python
 @app.route('/tasks/<int:task_id>', methods=['PUT'])
 def update_task(task_id):
     if not request.json:
         return jsonify({'error': 'Datos no proporcionados en formato JSON'}), 400

#     conn = get_db_connection()
#     cursor = conn.cursor()
    
#     # Verificar si la tarea existe
#     cursor.execute("SELECT * FROM tasks WHERE id = ?", (task_id,))
#     tarea_existente = cursor.fetchone()
#     if not tarea_existente:
#         conn.close()
#         return jsonify({'error': 'Tarea no encontrada'}), 404
        
#     # Obtener datos del request, usando los existentes si no se proporcionan nuevos
#     titulo_actualizado = request.json.get('titulo', tarea_existente['titulo'])
#     descripcion_actualizada = request.json.get('descripcion', tarea_existente['descripcion'])
#     # Para 'completada', si viene en el JSON, usarlo, sino, el valor existente.
#     # Convertir el booleano de Python a 0/1 para SQLite.
#     if 'completada' in request.json:
#         completada_actualizada_bool = request.json['completada']
#         if not isinstance(completada_actualizada_bool, bool):
#             conn.close()
#             return jsonify({'error': 'El campo "completada" debe ser un booleano (true/false)'}), 400
#         completada_actualizada_int = 1 if completada_actualizada_bool else 0
#     else:
#         completada_actualizada_int = tarea_existente['completada'] # Ya es 0 o 1
        
#     try:
#         cursor.execute(
#             "UPDATE tasks SET titulo = ?, descripcion = ?, completada = ? WHERE id = ?",
#             (titulo_actualizado, descripcion_actualizada, completada_actualizada_int, task_id)
#         )
#         conn.commit()
#     except sqlite3.Error as e:
#         conn.rollback()
#         conn.close()
#         return jsonify({'error': f'Error en la base de datos: {e}'}), 500
    
#     # Obtener la tarea actualizada para devolverla
#     cursor.execute("SELECT * FROM tasks WHERE id = ?", (task_id,))
#     tarea_actualizada_fila = cursor.fetchone()
#     conn.close()
    
#     tarea_actualizada = dict(tarea_actualizada_fila)
#     tarea_actualizada['completada'] = bool(tarea_actualizada['completada'])
    
#     return jsonify(tarea_actualizada)
```

#### e) Eliminar una Tarea (DELETE /tasks/<task_id>)

```python
# @app.route('/tasks/<int:task_id>', methods=['DELETE'])
# def delete_task(task_id):
#     conn = get_db_connection()
#     cursor = conn.cursor()
    
#     # Verificar si la tarea existe antes de intentar eliminar
#     cursor.execute("SELECT * FROM tasks WHERE id = ?", (task_id,))
#     if not cursor.fetchone():
#         conn.close()
#         return jsonify({'error': 'Tarea no encontrada'}), 404

#     try:
#         cursor.execute("DELETE FROM tasks WHERE id = ?", (task_id,))
#         conn.commit()
#     except sqlite3.Error as e:
#         conn.rollback()
#         return jsonify({'error': f'Error en la base de datos: {e}'}), 500
#     finally:
#         conn.close()
        
#     return jsonify({'mensaje': 'Tarea eliminada exitosamente'}), 200 # o 204 No Content
```

### 3.3. Código Completo de la API con SQLite (`tasks_api_sqlite.py`)

Aquí juntaríamos todas las partes en un solo script.

In [ ]:
# Este bloque contendría el código completo como se escribiría en un archivo .py
# (similar a juntar los fragmentos de 3.1 y 3.2)
# Por brevedad en este notebook, se omite la repetición completa aquí,
# pero se entiende que es la combinación de los snippets anteriores.

print("Imagina aquí el código completo del archivo tasks_api_sqlite.py...")
print("Asegúrate de incluir la llamada a init_db() antes de app.run() si es necesario.")

---

## 4. Ejecutando y Probando la API con Base de Datos

1.  Guarda el código completo de la API en un archivo, por ejemplo, `tasks_api_sqlite.py`.
2.  Ejecútalo desde tu terminal: `python tasks_api_sqlite.py`.
3.  Al ejecutarlo, se creará (si no existe) un archivo `tasks.db` en el mismo directorio. Este es tu archivo de base de datos SQLite.
4.  Utiliza Postman, Insomnia, `curl` o la librería `requests` de Python para probar los diferentes endpoints (GET, POST, PUT, DELETE).
5.  Verifica que los datos persisten incluso si detienes y reinicias el servidor Flask.
6.  Puedes inspeccionar el archivo `tasks.db` con una herramienta como "DB Browser for SQLite" para ver los datos directamente.

---

## 5. Consideraciones Adicionales

* **Seguridad contra Inyección SQL:** Al usar `cursor.execute("SQL...", (parametros,))` con `?` como placeholders, el módulo `sqlite3` se encarga de escapar los parámetros correctamente, previniendo la mayoría de los ataques de inyección SQL. Nunca construyas queries SQL concatenando strings directamente con datos de entrada del usuario.
* **Gestión de Conexiones en Flask:** Para aplicaciones más complejas, Flask ofrece patrones para gestionar conexiones a bases de datos de forma más eficiente (ej. usando el objeto `g` de Flask y `teardown_appcontext` para asegurar que las conexiones se cierren después de cada petición). Lo que hicimos (abrir/cerrar por función) es simple pero puede no ser lo más óptimo para alto tráfico.
* **Manejo de Errores:** Un manejo de errores más robusto (bloques `try-except-finally`) es crucial en producción.
* **Esquemas y Migraciones:** Si la estructura de tu tabla (`schema`) necesita cambiar después de que la aplicación esté en uso, necesitarás un sistema de "migraciones" para actualizar la base de datos sin perder datos (herramientas como Alembic pueden ayudar, especialmente con SQLAlchemy).
* **ORMs (Object-Relational Mappers):** Para interactuar con bases de datos de una manera más orientada a objetos y abstracta (sin escribir SQL directamente), se pueden usar ORMs como SQLAlchemy o Peewee. Flask tiene extensiones como `Flask-SQLAlchemy` que facilitan esto.

---

## 6. Resumen

Hemos aprendido cómo conectar una API Flask a una base de datos SQLite para lograr la persistencia de datos. Esto implica:
* Usar el módulo `sqlite3` de Python.
* Definir un esquema de tabla.
* Modificar los endpoints CRUD para ejecutar operaciones SQL (`INSERT`, `SELECT`, `UPDATE`, `DELETE`).
* Manejar conexiones y commits.

Esta es una habilidad fundamental para construir aplicaciones backend que manejen datos de manera efectiva.

## (Opcional) Ejercicio Práctico 🚀

Tomando el ejercicio opcional de la clase anterior donde creaste una API CRUD para "libros":

1.  Define un esquema para una tabla `libros` en SQLite. Debe tener al menos columnas para `id` (INTEGER PRIMARY KEY AUTOINCREMENT), `titulo` (TEXT NOT NULL), `autor` (TEXT NOT NULL), y `anio_publicacion` (INTEGER).
2.  Crea una función `init_libros_db()` para crear esta tabla.
3.  Adapta todos tus endpoints CRUD (GET todos, GET por ID, POST, PUT, DELETE) de la API de libros para que lean y escriban en la tabla `libros` de una base de datos SQLite (`libros.db`).
4.  Asegúrate de manejar las conexiones a la base de datos y los commits apropiadamente.
5.  Prueba tu API exhaustivamente.